# Variational Monte Carlo Demonstration
Authors: Hannah Baek, Tobias Tian

## About this Notebook
- In this notebook, we are going to demonstrate how we can use VMC to estimate ground-state energy of a many-body physical system.
- For demonstrating the plausibility of this algorithm, while maintaining simplicity, we decide to follow a classic example in showing the effectiveness of VMC, the helium atom.
- Without highlighting too much of the physical details which can overshadow the idea of VMC, we will restrict our attention to the electronic degrees of freedom of the helium atom. The nucleus, consisting of 2 protons and 2 neutrons, is treated as fixed within the Born-Oppenheimer approximation. This way, the system can be regarded as consisting 2 interacting electrons in the Coulomb potential of a single immobile nucleus. Introducing the nucleus into the system will drastically complicate this demonstration and shift the focus to physics instead of the method itself, as we need to consider the interaction within the nucleus.

## Background Information
- Our objective in this demonstration is to determine the electronic ground-state energy of the helium atom.
- We will be using a trial wavefunction that is dependent on the Cartesian co-ordinates of the two electrons $\mathbf{r}_1, \mathbf{r}_2$: $$\Psi_T(\mathbf{r}_1, \mathbf{r}_2)$$ and we call $\mathbf{R} = (\mathbf{r}_1, \mathbf{r}_2)$ to be a "configuration"
- The ground-state energy is approximated by: $$E_0 \approx \frac{\langle \Psi_T | \hat{H} | \Psi_T \rangle}{\langle \Psi_T | \Psi_T \rangle}$$
- However, the above approximation does not progress naturally, this is where we introduce VMC. We will be sampling according to the trial wavefunction $\Psi_T$ and obtain a list of configurations $[\mathbf{R}_1, \mathbf{R}_2, \dots, \mathbf{R}_n]$. For each of the configuration, we will calculate its local energy: $$E_{\text{loc}, i} = \frac{\hat{H}\Psi_T(\mathbf{R}_i)}{\Psi_T(\mathbf{R}_i)}$$ and take the average to approximate $$E_0 \approx \frac{1}{n}\sum_{i = 1}^{n} E_{\text{loc}, i}$$
- The Hamiltonian, in this helium atom example, $\hat{H}$, is given to be: $$\hat{H} = -\frac{1}{2}\nabla_1^2 - \frac{1}{2}\nabla_2^2 - \frac{2}{r_1} - \frac{2}{r_2} + \frac{1}{r_{12}}$$ where this is true in "atomic units".
- In "atomic units" system, we will be "defining" the constants of $\hbar$ (Planck's constant), $m_e$ (mass of an electron), $e$ (the charge of one electron), $4\pi\varepsilon_0$ (permittivity) to be $1$. The result of such "definitions" is that, length will now be measured in Bohr radii ($a_0$), and energy will now be measured in Hartree ($E_h$).

## Coding Demonstration

### Setup

- $Z$ is the atomic number, for Helium, we take $Z = 2$.
- Selecting $a_{\text{cusp}} = 0.5$ captures the cusp conditions of the electrons well.

In [1]:
# Setup, relevant parameters

import numpy as np
import random

Z = 2.0
a_cusp = 0.5
rng = np.random.default_rng(seed=random.randint(0, 255))

### Trial Wavefunction Selection

#### Slater Determinant
- For the ground state of helium both electrons occupy the same spatial $1s$ orbital with opposite spins. The spatial part of the Slater determinant therefore reduces to a product: $$\Phi(\mathbf{R}) = \phi_{1s}(\mathbf{r}_1)\phi_{1s}(\mathbf{r}_2)$$ where $\phi_{1s}(\mathbf{r}) \propto e^{-Z\mathbf{r}}$.
- The normalization constants cancel in Metropolis ratios and in the local energy, the implementation uses: $$\ln \Phi(\mathbf{R}) = -Z(|\mathbf{r}_1| + |\mathbf{r}_2|)$$
- This form satisfies the exact electron--nucleus cusp condition and gives an analytically simple expression for gradients and Laplacians, which reduces the variance of the local energy estimator.

#### Jastrow Factor
- To incorporate electron--electron correlation we adopt a simple two-body Jastrow factor of the form: $$J(\mathbf{R}, \beta) = \exp(u(\mathbf{r}_{12}, \beta)), u(\mathbf{r}_{12}, \beta) = \frac{a_{\text{cusp}} \mathbf{r}_{12}}{1 + \beta \mathbf{r}_{12}}$$ where $\mathbf{r}_{12} = |\mathbf{r}_1 - \mathbf{r}_2|$ and $a_{\text{cusp}} = \frac{1}{2}$ is fixed by the electron--electron cusp condition for opposite-spin electrons, $\beta$ is a variational parameter controlling the range of the correlation hole.
- The code therefore uses: $$\ln J(\mathbf{R}, \beta) = u(\mathbf{r}_{12}, \beta)$$ $$\ln \Psi_T(\mathbf{R}) = \ln \Phi(\mathbf{R}) + \ln J(\mathbf{R}, \beta)$$ This Jastrow is real-valued and preserves the antisymmetry provided by the Slater determinant.

In [2]:
# This function takes in relevant parameters, and a configuration,
# and outputs the calculated value of the wavefunction

def norm(v):
    return np.sqrt(np.dot(v, v))

def distances(R):
    # returns r_1, r_2, r_12
    r1 = norm(R[0])             # distance between 1st electron and the nucleus
    r2 = norm(R[1])             # distance between 2nd electron and the nucleus
    r12 = norm(R[0] - R[1])     # distance between the 2 electrons
    return r1, r2, r12

def log_slater(R):
    # Logarithm of Phi = exp[-Z(r_1+r_2)]
    r1, r2, _ = distances(R)
    return -Z * (r1 + r2)

def log_jastrow(R, beta):
    # Logarithm of Jastrow = u(r_12)
    _, _, r12 = distances(R)
    if r12 == 0:
        return 0.0
    return a_cusp * r12 / (1 + beta * r12)

def log_psi(R, beta):
    # Total log Psi_T
    return log_slater(R) + log_jastrow(R, beta)

def psi_sq(R, beta):
    # |Psi_T|^2 
    return np.exp(2 * log_psi(R, beta))

### Sampling the Trial Wavefunction (Metropolis Algorithm)

- The probability density $|\Psi_T(\mathbf{R}, \beta)|^2$ is sampled using the Metropolis algorithm.
- Start from a random initial configuration. For demonstration purposes, we propose the initial configuration to be $(0.5, 0, 0)$, $(0, 0.5, 0)$ for the electrons respectively with nucleus at $(0, 0, 0)$. The $0.5$ is chosen to represent $0.5$ Bohr radii away from from the nucleus.
- Propose $\mathbf{R}' = \mathbf{R} + \delta \mathbf{R}$ with $\delta \mathbf{R}$ drawn from a uniform distribution with width `step_size`.
- Accept the move with probability:
$$P := \min(1, \frac{|\Psi_T(\mathbf{R}', \beta)|^2}{|\Psi_T(\mathbf{R}, \beta)|^2})$$
- The function `metropolis` implements this procedure and after certain steps, the local energy is recorded at each accepted configuration.

In [3]:
# This function takes in the trial wavefunction selected,
# and outputs a list of "accepted" configurations

def initial_R():
    return np.array([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0]], dtype=float)

def metropolis(beta, n_steps, step_size=0.1):
    R = initial_R()
    psi2_old = psi_sq(R, beta)

    samples = [R.copy()]

    for step in range(1, n_steps):

        # Uniform random displacement in [-step_size/2, +step_size/2]
        dR = rng.uniform(
            low=-step_size/2,
            high= step_size/2,
            size=R.shape
        )

        R_new = R + dR
        psi2_new = psi_sq(R_new, beta)

        P = psi2_new / psi2_old

        if P >= 1 or rng.random() < P:
            R = R_new
            psi2_old = psi2_new
            samples.append(R.copy())

    return samples


### Calculating Local Energy

- Recall that we calculate the local energy with $$E_{\text{loc}}(\mathbf{R}) = \frac{\hat{H}\Psi_T(\mathbf{R}, \beta)}{\Psi_T(\mathbf{R}, \beta)}$$
- For the given Hamiltonian as described above, we can compute the local energy to be: $$E_{\text{loc}, \beta} = -\frac{1}{2}\sum_{i = 1}^2 (\nabla_i^2 \ln \Psi_T + |\nabla_i \ln \Psi_T|^2) - Z(\frac{1}{|\mathbf{r}_1|} + \frac{1}{|\mathbf{r}_2|}) + \frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|}$$ where $Z$ is the atomic number, and $Z = 2$.
- To reduce variance, the derivatives of $\ln \Psi_T$ are computed analytically.
- For the Slater part, we have $$\nabla_1 \ln \Phi = -Z \hat{\mathbf{r}}_1, \nabla_2 \ln \Phi = -Z \hat{\mathbf{r}}_2$$ $$\nabla_1^2 \ln \Phi = -\frac{2Z}{|\mathbf{r}_1|}, \nabla_2^2 \ln \Phi = -\frac{2Z}{|\mathbf{r}_2|}$$
- For the Jastrow form ($\mathbf{r} = \mathbf{r}_{12}$), we have $$u'(\mathbf{r})=\frac{a_{\text{cusp}}}{(1 + \beta \mathbf{r})^2}, u''(\mathbf{r})=-\frac{2a_{\text{cusp}}}{(1 + \beta \mathbf{r})^3}$$ $$\nabla_1 u = u'(\mathbf{r}_{12})\hat{s}, \nabla_2 u = -u'(\mathbf{r}_{12})\hat{s}$$ $$\nabla_1^2 u = \nabla_2^2 u = u''(\mathbf{r}_{12}) + \frac{2}{\mathbf{r}_{12}}u'(\mathbf{r}_{12})$$ where $s = \mathbf{r}_1 - \mathbf{r}_2$ and $\hat{s} = \frac{s}{|s|}$
- Since $\ln \Psi_T = \ln \Phi + u$, we must have $$\nabla_i \ln \Psi_T = \nabla_i \ln \Phi + \nabla_i u$$ so $$|\nabla_i \ln \Psi_T|^2 = |\nabla_i \ln \Phi|^2 + |\nabla_i u|^2 + 2 \nabla_i \ln \Phi \cdot \nabla_i u$$ and $$\nabla_i^2 \ln \Psi_T = \nabla_i^2 \ln \Phi + \nabla_i^2 u$$
- The kinetic energy is formed from these analytic derivatives, while the potential energy uses the standard electron--nucleus and electron--electron Coulomb interactions.

In [4]:
# This function takes in a configuration from the list of samples obtained,
# and outputs the local energy at that configuration

def grad_log_psi(R, beta):
    r1_vec, r2_vec = R
    s = r1_vec - r2_vec

    r1 = norm(r1_vec)
    r2 = norm(r2_vec)
    r12 = norm(s)

    r1_hat = r1_vec/r1 if r1 > 0 else np.zeros(3)
    r2_hat = r2_vec/r2 if r2 > 0 else np.zeros(3)
    s_hat  = s/r12 if r12 > 0 else np.zeros(3)

    g1 = -Z * r1_hat
    g2 = -Z * r2_hat

    if r12 > 0:
        up = a_cusp / (1 + beta*r12)**2
        g1 += up * s_hat
        g2 -= up * s_hat

    return g1, g2

def laplacian_log_psi(R, beta):
    r1_vec, r2_vec = R
    s = r1_vec - r2_vec

    r1 = norm(r1_vec)
    r2 = norm(r2_vec)
    r12 = norm(s)

    lap1 = -2*Z/r1 if r1 > 0 else 0
    lap2 = -2*Z/r2 if r2 > 0 else 0

    if r12 > 0:
        up = a_cusp / (1 + beta*r12)**2
        upp = -2*a_cusp*beta/(1 + beta*r12)**3
        lap_u = upp + (2/r12)*up
        lap1 += lap_u
        lap2 += lap_u

    return lap1 + lap2

def energy_local(R, beta):
    r1, r2, r12 = distances(R)
    g1, g2 = grad_log_psi(R, beta)
    lap = laplacian_log_psi(R, beta)

    kinetic = -0.5 * (lap + np.dot(g1, g1) + np.dot(g2, g2))
    potential = -Z*(1/r1 + 1/r2) + 1/r12

    return kinetic + potential

### Approximating Ground-state Energy

In [5]:
# This function takes in a list of local energies,
# and outputs the sample average to approximate ground-state energy

def ground_state(local_energies):
    local_energies = np.array(local_energies)
    return np.mean(local_energies), np.var(local_energies)

### Optimizing the Trial Wavefunction

In [6]:
# This function scans over beta and returns the one with minimum variance

def optimize(beta_grid, n_steps):
    results = []
    best_beta = None
    best_var = np.inf

    for beta in beta_grid:      # for each beta, do a VMC, and pick the beta with the least variance associated
        samples = metropolis(beta, n_steps)
        energies = [energy_local(R, beta) for R in samples]
        meanE, varE = ground_state(energies)

        results.append((beta, meanE, varE))
        if varE < best_var:
            best_var = varE
            best_beta = beta

    return best_beta, results

### Main Process

In [7]:
# This function represents the main process of one round of VMC

def main(beta, n_steps):
    samples = metropolis(beta, n_steps)
    energies = [energy_local(R, beta) for R in samples]
    meanE, varE = ground_state(energies)
    return meanE, varE, energies

In [8]:
meanE, varE, energies = main(beta = 0.3, n_steps = 500000)

print(f"Estimated ground-state energy: = {meanE:.6f} Hartree")
print(f"Variance of local energy: = {varE:.6f}")

Estimated ground-state energy: = -2.874865 Hartree
Variance of local energy: = 0.084343


### Iterations of the Process

In [9]:
#Optimizing beta parameter
beta_grid = np.linspace(0.0, 3.0, 20)
best_beta, results = optimize(beta_grid, n_steps=10000)     # select the best beta from a short metropolis sampling

meanE_bestbeta, varE_bestbeta, energies = main(best_beta, n_steps=500000)   # compute the ground state energy with the best beta obtained, with longer metropolis steps
print(f"Estimated ground-state energy: = {meanE_bestbeta:.6f} Hartree")
print(f"Variance of local energy: = {varE_bestbeta:.6f}")

Estimated ground-state energy: = -2.881377 Hartree
Variance of local energy: = 0.082461


### Results

In [10]:
import pandas as pd

def summary_table_from_results(
    first_beta,
    first_mean,
    first_var,
    best_beta,
    opt_mean,
    opt_var
):
    exact_energy = -2.9037
    hf_energy = -2.8617

    df = pd.DataFrame({
        "Description": [
            "First iteration (initial β)",
            "Optimized β",
            "Exact helium ground-state energy",
            "Hartree–Fock energy"
        ],
        "β value": [
            first_beta,
            best_beta,
            "N/A",
            "N/A"
        ],
        "Energy (Hartree)": [
            first_mean,
            opt_mean,
            exact_energy,
            hf_energy
        ],
        "Variance": [
            first_var,
            opt_var,
            "N/A",
            "N/A"
        ]
    })
    
    return df
    
summary_table_from_results(
    first_beta = 0.3,
    first_mean = meanE,
    first_var = varE,
    best_beta = best_beta,
    opt_mean = meanE_bestbeta,
    opt_var = varE_bestbeta
)

,Description,β value,Energy (Hartree),Variance
0,First iteration (initial β),0.3,-2.874865,0.084343
1,Optimized β,0.315789,-2.881377,0.082461
2,Exact helium ground-state energy,N/A,-2.903700,N/A
3,Hartree–Fock energy,N/A,-2.861700,N/A


The code above is written with the aid of ChatGPT for debugging purposes.

It must be noted the above implementation of the VMC is still a rather naive version, as this implementation fails to consider other relevant statistical qualities. For example, for a better VMC, one could "throw away" the first configurations until the configuration becomes relatively stable, and monitor the acceptance rate. This demonstration does none of that. The key of this demonstration is show how VMC works structurally in code, and highlight that even with many inefficiencies, it is possible to already use it for small many-body systems like a helium atom.